# 4. Spatial Smoothing, Regionalization, and Neighborhood Analysis
## 4.1 Spatial Smoothing
#### 4.1.1 Introduction
In many studies the target variable may need to modeled using a rate or normalized value due to a number of reasons including:
* Varying underlying population
* Variation in the age structure of underlying population
* Variation in environmental variables across a study area
* Arbitrarily shaped areal units

These rates can be heavily influenced by high or low raw counts. In order to reduce the effect of high raw value counts on the rates, spatial smoothing can be used to moderate extreme values at an observation.

#### 4.1.2 Mean and Median Smoothing
One of the simplest ways to conduct spatial smoothing is by using locally weighted smoothing. This technique takes values from surrounding areal units (based on the W matrix) and uses a weighted average of the weights to produce a value at a given observation.

In [5]:
# Conduct mean smoothing on a dataset
import pysal
import numpy as np
from pysal.esda import smoothing as sm
# tw = pysal.queen_from_shapefile("./data/census/stpete_cenacs_2015.shp") # W
w = pysal.open('./data/census/stpete_cenacs_2015.gal', 'r').read()
cenacs = pysal.open("./data/census/stpete_cenacs_2015.dbf", 'r')
e, b = np.array(cenacs[:,58]), np.array(cenacs[:,15])
if not w.id_order_set: w.id_order = range(1,len(cenacs) + 1)
rate = sm.Disk_Smoother(e, b, w)
rate.r

/home/tankofvines/anaconda3/lib/python3.5/site-packages/pysal/esda/smoothing.py:1223: RuntimeWarning: invalid value encountered in true_divide
  r = e * 1.0 / b


array([[        nan],
       [ 0.00756632],
       [ 0.00846333],
       [ 0.00867053],
       [ 0.00784545],
       [ 0.00934544],
       [ 0.00945583],
       [ 0.00622045],
       [ 0.00784219],
       [ 0.00727587],
       [ 0.00719841],
       [ 0.00654591],
       [ 0.00668667],
       [ 0.00838505],
       [ 0.00801984],
       [ 0.00286353],
       [ 0.00640873],
       [ 0.007038  ],
       [ 0.00727541],
       [ 0.00749259],
       [ 0.00851467],
       [ 0.00862775],
       [ 0.00819592],
       [ 0.00504671],
       [ 0.00718706],
       [ 0.00466764],
       [ 0.00651226],
       [ 0.00665815],
       [ 0.00612909],
       [ 0.00261204],
       [ 0.00845143],
       [ 0.00438472],
       [        nan],
       [ 0.00623177],
       [ 0.01502839],
       [        nan],
       [        nan],
       [ 0.00582524],
       [ 0.0087826 ],
       [ 0.00543296],
       [ 0.01240989],
       [ 0.00601352],
       [ 0.00428964],
       [ 0.00413996],
       [ 0.00369789],
       [ 0

In [3]:
rate = sm.Spatial_Median_Rate(e, b, w)
rate.r

/home/tankofvines/anaconda3/lib/python3.5/site-packages/pysal/esda/smoothing.py:1331: RuntimeWarning: invalid value encountered in true_divide
  self.r = e * 1.0 / b
/home/tankofvines/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


array([        nan,  0.00833893,  0.00756902,  0.00768666,  0.00686135,
        0.00795688,  0.00798899,  0.00763323,  0.00482053,  0.00770307,
        0.00754005,  0.00566666,  0.00630112,  0.00652539,  0.00765884,
        0.00414698,  0.00493584,  0.00458553,  0.00701823,  0.00658659,
        0.00664778,  0.00758495,  0.00626882,  0.00449771,  0.00245608,
        0.00363525,  0.00628846,  0.00607685,  0.00402761,  0.00212203,
        0.00664778,  0.00482053,         nan,  0.00534467,  0.01083487,
               nan,         nan,  0.00476019,  0.00752355,  0.0035343 ,
        0.00939198,  0.00592914,  0.00361656,  0.00423522,         nan,
        0.00592914,  0.01260488,  0.00484802,  0.00628846,  0.0012956 ,
        0.0012956 ,  0.00626882,  0.00659877,  0.00259557,  0.00598634,
        0.00575982,  0.00575982,  0.00447252,  0.00292844,  0.00556894,
        0.00291713,  0.00678425,  0.00565538,  0.00254517,  0.00197972,
        0.00246467,  0.00129375,  0.00607835,  0.0021732 ,  0.00

#### 4.1.3 Spatial Missing Value Imputation


## 4.2 Regionalization
#### 4.2.1 Introduction
Spatial regions are groups of observations that share similar qualities to other observations they share a boundary with. Regionalization is a technique used assign regions to a spatial dataset and attribute region membership to the observation. The technique is analagous to unsupervised learning methods such as hierarchical or k-means clustering that determine cluster membership in feature space.
#### 4.2.2 max-p
Max-p is a regionalization method that does not require the number of regions (clusters) apriori. The only required values are a floor constraint for the minimum number of observations within a region. Let's calculate max-p for the median household income values in St. Petersburg, FL.

In [5]:
# Calculate max-p for a set of areal units
stnorm = pysal.open("./data/stpete_cenacs_2014_norms.dbf", 'r')
y = np.array(stnorm)
r = pysal.Maxp(w, y, floor = 5, floor_variable = np.ones((231, 1)), initial = 99)
r.regions

[['39', '105', '64', '121', '62', '186', '182'],
 ['145', '90', '146', '23', '28', '35', '79', '46', '147', '11', '86', '48'],
 ['47', '148', '123', '1', '125', '77', '88', '165', '59', '133', '63', '91'],
 ['172', '215', '155', '214', '225'],
 ['200', '49', '193', '196', '3'],
 ['131', '189', '71', '31', '117'],
 ['113', '37', '38', '205', '50', '107'],
 ['44', '45', '43', '194', '192', '140', '98', '132', '106'],
 ['2', '144', '190', '52', '149'],
 ['87', '89', '94', '96', '93'],
 ['99', '16', '14', '97', '12'],
 ['27', '219', '60', '9', '58'],
 ['68', '72', '67', '73', '75', '136'],
 ['227', '61', '83', '102', '84'],
 ['198', '54', '203', '211', '204'],
 ['20', '108', '34', '82', '25', '17', '81', '122', '195'],
 ['158', '224', '159', '134', '201'],
 ['51', '6', '181', '212', '80'],
 ['66', '175', '177', '230', '151', '226'],
 ['120', '70', '119', '124', '0'],
 ['222', '223', '130', '138', '26', '173', '163', '110', '167'],
 ['76', '4', '114', '116', '69'],
 ['53', '65', '168', '166

## 4.3 Neighborhood Analysis